<a href="https://colab.research.google.com/github/YOON00207/inquiry-sentiment-analysis/blob/main/%ED%8D%BC%EC%8A%A4%ED%8A%B8%EB%B0%B8%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 패키지 설치 및 라이브러리 불러오기

In [1]:
pip install pandas scikit-learn sentence-transformers openpyxl


In [2]:
pip install matplotlib

In [5]:
pip install transformers emoji pandas matplotlib accelerate


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 608.4/608.4 kB 7.8 MB/s eta 0:00:00


In [4]:
import re
import html
import emoji
import pandas as pd
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from google.colab import drive



ModuleNotFoundError: No module named 'emoji'

## 데이터 블러오기

In [ ]:
drive.mount('/content/drive')

import pandas as pd

file_path = '/content/drive/MyDrive/2025_문제해결은행/퍼스트밸류/마켓문의_20000건_1.xlsx'
df = pd.read_excel(file_path, header =1)
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,market_account_no,market_inquiry_no,inquiry_type_cd,inquiry_type,inquiry_title,inquiry_content,inquiry_date,reply_yn,inquiry_raw,update_dt,create_dt
0,391,217213738,N,미분류,아고가 파푸아뉴기니 블루마운틴 커피 1kg 홀빈/분쇄,안녕하세요:)\r\n해당 커피 맛있게 먹고있는데\r\n혹시 상품 언제 재입고 되는지...,2023-09-11 19:56:27.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":34087673,""deal_srl"":...",2023-09-14 08:30:49.000,2023-09-12 01:30:47.000
1,394,217681990,N,미분류,[티나는추석] 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 선물세트 + 쇼핑백포함,어제 제품 받았는데요<br>4개씩 총 6건 주문했는데 <br>플러스1까지 30개인데...,2023-09-23 08:59:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":33221833,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000
2,394,217690754,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,오늘 사은품만 받았어요 \r\n본품 4박스는 언제 배송되는지 확인 부탁드립니다,2023-09-23 15:02:19.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":9659193,""deal_srl"":1...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000
3,394,217691782,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,3번 세트 4개 구입했는데요.\r\n오늘..사은품인 녹용골드스틱만 오고 배송완료라고...,2023-09-23 15:46:28.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":13678098,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000
4,394,217692250,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,300포 주문했는데..30포가 왔네요?<br>큰박스에 달랑 세 뭉치 들어 깜놀이었...,2023-09-23 16:08:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":18444189,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000


## 텍스트 전처리

In [ ]:
#텍스트 전처리
def remove_html(text):
    return re.sub(r"<.*?>", " ", text)

def clean_newlines(text):
    return text.replace("\\r", " ").replace("\\n", " ").replace("\n", " ").replace("\t", " ")

def remove_emoji(text):
    return emoji.replace_emoji(text, "")

def remove_url(text):
    return re.sub(r"http\S+|www\S+", "", text)

def remove_special(text):
    # 한글, 영어, 숫자, 감정 표현부호 ?!, . , 만 허용
    return re.sub(r"[^가-힣0-9a-zA-Z?!.,\s]", " ", text)

def clean_spaces(text):
    return ' '.join(text.split())

def remove_greetings(text):
    greetings = [
        "안녕하세요", "첨부이미지"
    ]
    for g in greetings:
        text = text.replace(g, " ")
    return text


def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    text = html.unescape(text)
    text = remove_html(text)
    text = remove_url(text)
    text = clean_newlines(text)
    text = remove_emoji(text)
    text = remove_special(text)
    text = clean_spaces(text)
    text = remove_greetings(text)
    return text.strip()


In [ ]:
# 문의글 컬럼명 변경 가능 — 여기 맞춰 주세요
text_col = "inquiry_content"

# 전처리 적용
df["clean_text"] = df[text_col].fillna("").apply(preprocess_text)

df.head()

,market_account_no,market_inquiry_no,inquiry_type_cd,inquiry_type,inquiry_title,inquiry_content,inquiry_date,reply_yn,inquiry_raw,update_dt,create_dt,clean_text
0,391,217213738,N,미분류,아고가 파푸아뉴기니 블루마운틴 커피 1kg 홀빈/분쇄,안녕하세요:)\r\n해당 커피 맛있게 먹고있는데\r\n혹시 상품 언제 재입고 되는지...,2023-09-11 19:56:27.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":34087673,""deal_srl"":...",2023-09-14 08:30:49.000,2023-09-12 01:30:47.000,해당 커피 맛있게 먹고있는데 혹시 상품 언제 재입고 되는지 알 수 있을까요!? 재입...
1,394,217681990,N,미분류,[티나는추석] 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 선물세트 + 쇼핑백포함,어제 제품 받았는데요<br>4개씩 총 6건 주문했는데 <br>플러스1까지 30개인데...,2023-09-23 08:59:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":33221833,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,어제 제품 받았는데요 4개씩 총 6건 주문했는데 플러스1까지 30개인데요 실제 25...
2,394,217690754,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,오늘 사은품만 받았어요 \r\n본품 4박스는 언제 배송되는지 확인 부탁드립니다,2023-09-23 15:02:19.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":9659193,""deal_srl"":1...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,오늘 사은품만 받았어요 본품 4박스는 언제 배송되는지 확인 부탁드립니다
3,394,217691782,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,3번 세트 4개 구입했는데요.\r\n오늘..사은품인 녹용골드스틱만 오고 배송완료라고...,2023-09-23 15:46:28.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":13678098,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,3번 세트 4개 구입했는데요. 오늘..사은품인 녹용골드스틱만 오고 배송완료라고 떴어...
4,394,217692250,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,300포 주문했는데..30포가 왔네요?<br>큰박스에 달랑 세 뭉치 들어 깜놀이었...,2023-09-23 16:08:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":18444189,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,300포 주문했는데..30포가 왔네요? 큰박스에 달랑 세 뭉치 들어 깜놀이었네요.....


## 모델 로드

In [ ]:
#실험 1 0: 부정 1: 중립 2: 긍정
#성능 BAD PASS
# 모델 및 토크나이저 로드
# model_name = "alsgyu/sentiment-analysis-fine-tuned-model"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

In [ ]:
# model_name = "jhgan/ko-sroberta-multitask"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name)
# model.to("cuda" if torch.cuda.is_available() else "cpu")


tokenizer_config.json:   0%|          | 0.00/585 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/744 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/443M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at jhgan/ko-sroberta-multitask and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [ ]:
# model_name = "beomi/KcBERT-base"

# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
# model.to("cuda" if torch.cuda.is_available() else "cpu")

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at beomi/KcBERT-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
model_name = "hun3359/klue-bert-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
model.to("cuda" if torch.cuda.is_available() else "cpu")

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## 감성분석

In [ ]:
def predict_sentiments(texts, batch_size=16):
    results = []
    device = "cuda" if torch.cuda.is_available() else "cpu"

    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]

        inputs = tokenizer(
            batch,
            padding=True,
            truncation=True,
            max_length=256,
            return_tensors="pt"
        ).to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            preds = torch.argmax(logits, dim=-1).cpu().numpy()

        results.extend(preds)
    return results


In [ ]:
model.config.id2label

{0: '분노',
 1: '툴툴대는',
 2: '좌절한',
 3: '짜증내는',
 4: '방어적인',
 5: '악의적인',
 6: '안달하는',
 7: '구역질 나는',
 8: '노여워하는',
 9: '성가신',
 10: '슬픔',
 11: '실망한',
 12: '비통한',
 13: '후회되는',
 14: '우울한',
 15: '마비된',
 16: '염세적인',
 17: '눈물이 나는',
 18: '낙담한',
 19: '환멸을 느끼는',
 20: '불안',
 21: '두려운',
 22: '스트레스 받는',
 23: '취약한',
 24: '혼란스러운',
 25: '당혹스러운',
 26: '회의적인',
 27: '걱정스러운',
 28: '조심스러운',
 29: '초조한',
 30: '상처',
 31: '질투하는',
 32: '배신당한',
 33: '고립된',
 34: '충격 받은',
 35: '가난한 불우한',
 36: '희생된',
 37: '억울한',
 38: '괴로워하는',
 39: '버려진',
 40: '당황',
 41: '고립된(당황한)',
 42: '남의 시선을 의식하는',
 43: '외로운',
 44: '열등감',
 45: '죄책감의',
 46: '부끄러운',
 47: '혐오스러운',
 48: '한심한',
 49: '혼란스러운(당황한)',
 50: '기쁨',
 51: '감사하는',
 52: '신뢰하는',
 53: '편안한',
 54: '만족스러운',
 55: '흥분',
 56: '느긋',
 57: '안도',
 58: '신이 난',
 59: '자신하는'}

In [ ]:
df2 = df[:100]

In [ ]:
texts = df2["clean_text"].tolist()
df2["sentiment_raw"] = predict_sentiments(texts)
df2.head()


/tmp/ipython-input-3746522049.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["sentiment_raw"] = predict_sentiments(texts)


,market_account_no,market_inquiry_no,inquiry_type_cd,inquiry_type,inquiry_title,inquiry_content,inquiry_date,reply_yn,inquiry_raw,update_dt,create_dt,clean_text,sentiment_raw
0,391,217213738,N,미분류,아고가 파푸아뉴기니 블루마운틴 커피 1kg 홀빈/분쇄,안녕하세요:)\r\n해당 커피 맛있게 먹고있는데\r\n혹시 상품 언제 재입고 되는지...,2023-09-11 19:56:27.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":34087673,""deal_srl"":...",2023-09-14 08:30:49.000,2023-09-12 01:30:47.000,해당 커피 맛있게 먹고있는데 혹시 상품 언제 재입고 되는지 알 수 있을까요!? 재입...,6
1,394,217681990,N,미분류,[티나는추석] 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 선물세트 + 쇼핑백포함,어제 제품 받았는데요<br>4개씩 총 6건 주문했는데 <br>플러스1까지 30개인데...,2023-09-23 08:59:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":33221833,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,어제 제품 받았는데요 4개씩 총 6건 주문했는데 플러스1까지 30개인데요 실제 25...,29
2,394,217690754,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,오늘 사은품만 받았어요 \r\n본품 4박스는 언제 배송되는지 확인 부탁드립니다,2023-09-23 15:02:19.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":9659193,""deal_srl"":1...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,오늘 사은품만 받았어요 본품 4박스는 언제 배송되는지 확인 부탁드립니다,56
3,394,217691782,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,3번 세트 4개 구입했는데요.\r\n오늘..사은품인 녹용골드스틱만 오고 배송완료라고...,2023-09-23 15:46:28.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":13678098,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,3번 세트 4개 구입했는데요. 오늘..사은품인 녹용골드스틱만 오고 배송완료라고 떴어...,57
4,394,217692250,N,미분류,[제철푸드] 4+1 풍년보감 6년근 고려홍삼정 에브리데이 골드 홍삼스틱 10g x ...,300포 주문했는데..30포가 왔네요?<br>큰박스에 달랑 세 뭉치 들어 깜놀이었...,2023-09-23 16:08:36.000,Y,"{""deal_buy_yn"":""Y"",""m_no"":18444189,""deal_srl"":...",2023-09-26 08:31:05.000,2023-09-26 01:31:03.000,300포 주문했는데..30포가 왔네요? 큰박스에 달랑 세 뭉치 들어 깜놀이었네요.....,40


In [ ]:
df3['sentiment_raw'].unique()

array([ 6, 29, 56, 57, 40, 10, 37, 53, 42, 49, 46, 27, 55, 22, 18, 28, 35,
        0, 50, 25, 45, 11, 51])

In [ ]:
df3[df3['sentiment_raw'] == 49]

,clean_text,sentiment_raw
33,저번에 산거 사용하려고 뜯었는데 옆선이 입히는 와중에 계속 뜯어집니다 처음엔 그냥 ...,49
57,특대형사이즈는 왔는데 대형사이즈가 안 왔어요,49
62,주문한 기저귀가 1박스만 왔는데 1박스는 따로 오는게 맞을까요?,49
92,특대랑 점보 섞어 주문시 특대형 예약발송날짜에 같이 오는걸까요?,49


In [ ]:
df3 = df2[['clean_text','sentiment_raw']]

In [ ]:
df3.head(30)

,clean_text,sentiment_raw
0,해당 커피 맛있게 먹고있는데 혹시 상품 언제 재입고 되는지 알 수 있을까요!? 재입...,0
1,어제 제품 받았는데요 4개씩 총 6건 주문했는데 플러스1까지 30개인데요 실제 25...,0
2,오늘 사은품만 받았어요 본품 4박스는 언제 배송되는지 확인 부탁드립니다,0
3,3번 세트 4개 구입했는데요. 오늘..사은품인 녹용골드스틱만 오고 배송완료라고 떴어...,0
4,300포 주문했는데..30포가 왔네요? 큰박스에 달랑 세 뭉치 들어 깜놀이었네요.....,0
5,추석 선물로 주문했습니다 추석 전에 빠른 배송 받고 싶습니다 추석 전날까지 빠른 배...,0
6,추석 선물로 의미있는 구매라서 추석 연휴전날까지는 배송을 받고 싶습니다만 도착 예정...,0
7,23일날 구입해도 추석 전에 받을수 있는거죵??,0
8,3번제품 3개 5번제품 3개 주문했읍니다 사은품60개 누락안되게 잘 보내주시길 부탁...,0
9,24일에 추석전에 받을수있을까요? 안돼면 취소하려고합니다,1


In [ ]:
test_sentences = [
    "너무 좋아요 완전 만족합니다",     # ➜ 긍정
    "그냥 그렇네요 보통이에요",         # ➜ 중립
    "진짜 짜증나요 불량이에요 최악입니다"  # ➜ 부정
]

inputs = tokenizer(test_sentences, return_tensors="pt", padding=True, truncation=True)
with torch.no_grad():
    outputs = model(**inputs)

print(outputs.logits)
print(torch.argmax(outputs.logits, dim=1))


tensor([[-0.3189, -0.1398, -0.9447,  0.1076, -0.7067, -0.2232, -1.0427, -0.2145,
         -0.8921, -0.4748, -0.3166, -0.4429, -1.7091, -0.4980, -0.5093, -0.1659,
         -0.4872,  0.9398, -0.5769, -1.3159, -0.1166, -1.4747, -1.3910, -0.8999,
         -0.3236, -0.8831, -0.6081, -0.5856, -1.1859, -0.4765, -0.8731, -0.7028,
          0.1720, -0.1601, -1.1010, -0.9688, -0.4001, -0.0832, -0.9372,  0.5597,
         -0.5697, -1.1306, -0.7737, -1.0172, -1.6481, -0.4134,  0.8612, -1.5997,
         -2.2341, -0.6054,  3.5421,  3.0593,  3.3420,  3.8690,  7.9563,  2.9380,
          3.3752,  2.7907,  2.6059,  1.3990],
        [ 1.0794,  0.5159, -0.1824,  0.5582,  2.1587,  1.1126, -1.9579,  0.6401,
         -0.0539,  0.2482, -0.0891,  0.0392, -1.2603, -0.3791, -0.5736,  0.5870,
          0.6443, -0.5374,  0.5532,  0.8419, -0.4262, -0.2564, -0.8441, -0.0905,
          0.0637,  0.4707,  1.0449, -0.5539, -0.3395, -1.0479, -0.1796,  0.5232,
          0.2009,  0.8385, -0.4621, -0.9636, -0.3959,  0.5908, 